In [4]:
# Import necessary libraries
import pandas as pd
import numpy as np
import xarray as xr
import glob

# Define the file path and search for .nc files
path = "*.nc"
files = glob.glob(path)

# Define the frequency range (adjust if using full range)
start_period = 1  # Minimum frequency
end_period = 40  # Maximum frequency

# Initialize the final result DataFrame
final_result = pd.DataFrame(columns=["Time", "Hs", "Tz", "Tm", "Te", "WP"])

# Loop through each file in the list of .nc files
for file in files:
    # Open the netCDF file
    data = xr.open_dataset(file)
    
    # Extract variables from the dataset
    times = data.time
    S = data.SmaxXpsd
    frq = data.Frequency

    # Convert the Frequency data to a pandas Series
    frq = pd.Series(frq.values.squeeze())
    df = frq.diff().dropna()  # Difference between adjacent frequency values
    
    # Calculate rolling mean of frequency (average one value with the next one)
    f = frq.rolling(window=2).mean().dropna()

    # Loop through each time step
    for t in times:
        # Select the spectral moment for the current time
        spec = pd.Series(S.sel(time=t).values.squeeze())
        s = spec.rolling(window=2).mean().dropna()

        # Calculate moments by integrating over the frequency range
        ss0 = df * s
        ss1 = f * s * df
        ss2 = f**2 * s * df
        ss_1 = f**-1 * s * df

        # Calculate wave parameters
        m0 = ss0.sum()  # Zeroth moment
        m1 = ss1.sum()  # First moment
        m2 = ss2.sum()  # Second moment
        m_1 = ss_1.sum()  # Negative first moment

        Hs = 4 * np.sqrt(m0)  # Significant wave height
        Tz = np.sqrt(m0 / m2)  # Zero-crossing period
        Tm = m0 / m1  # Mean wave period
        Te = m_1 / m0  # Energy wave period
        WP = 0.49*Hs**2*Te  # Wave power

        # Create a row for this time step's results
        result = pd.DataFrame({
            "Time": [pd.to_datetime(t.values)],
            "Hs": [Hs.round(2)],
            "Tz": [Tz.round(2)],
            "Tm": [Tm.round(2)],
            "Te": [Te.round(2)],
            "WP": [WP.round(2)]
        })

        # Append the results for this time step to the final result DataFrame
        print(result)
        final_result = pd.concat([final_result, result], ignore_index=True)

# Output the final result DataFrame
print(final_result)


                 Time    Hs    Tz    Tm    Te    WP
0 2010-09-01 00:28:00  1.52  5.32  6.06  7.82  8.88
                 Time    Hs    Tz    Tm    Te     WP
0 2010-09-01 00:58:00  1.59  5.47  6.25  8.08  10.01
                 Time    Hs    Tz    Tm    Te    WP
0 2010-09-01 01:28:00  1.49  5.39  6.07  7.63  8.31
                 Time    Hs    Tz    Tm    Te    WP
0 2010-09-01 01:58:00  1.61  5.45  6.08  7.59  9.62
                 Time    Hs    Tz    Tm    Te    WP
0 2010-09-01 02:28:00  1.62  5.51  6.14  7.61  9.77
                 Time    Hs    Tz    Tm    Te    WP
0 2010-09-01 02:58:00  1.59  5.45  6.13  7.73  9.63
                 Time    Hs    Tz    Tm    Te    WP
0 2010-09-01 03:28:00  1.59  5.61  6.33  7.97  9.82
                 Time    Hs    Tz    Tm    Te    WP
0 2010-09-01 03:58:00  1.54  5.36  5.96  7.38  8.59
                 Time    Hs    Tz   Tm   Te    WP
0 2010-09-01 04:28:00  1.46  5.34  6.0  7.6  7.97
                 Time    Hs    Tz   Tm    Te    WP
0 2010-09-01 04

/tmp/ipykernel_2021/4069023272.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_result = pd.concat([final_result, result], ignore_index=True)


                 Time    Hs    Tz    Tm     Te    WP
0 2010-09-02 16:28:00  0.92  7.33  8.66  11.16  4.58
                 Time    Hs    Tz    Tm     Te    WP
0 2010-09-02 16:58:00  0.98  7.09  8.18  10.38  4.92
                 Time    Hs    Tz    Tm     Te   WP
0 2010-09-02 17:28:00  0.97  7.16  8.32  10.67  4.9
                 Time    Hs    Tz    Tm     Te    WP
0 2010-09-02 17:58:00  0.86  6.92  8.11  10.49  3.77
                 Time    Hs    Tz    Tm    Te    WP
0 2010-09-02 18:28:00  0.96  7.59  8.98  11.4  5.19
                 Time   Hs    Tz    Tm     Te    WP
0 2010-09-02 18:58:00  0.9  7.58  8.95  11.24  4.51
                 Time    Hs   Tz    Tm     Te    WP
0 2010-09-02 19:28:00  0.94  7.4  8.71  10.97  4.73
                 Time    Hs    Tz    Tm     Te    WP
0 2010-09-02 19:58:00  0.93  7.05  8.27  10.62  4.47
                 Time   Hs    Tz    Tm     Te    WP
0 2010-09-02 20:28:00  0.9  6.98  8.21  10.71  4.23
                 Time    Hs    Tz    Tm     Te    WP
0 2